In [2]:
import MetaTrader5 as mt5

# Connect to MetaTrader 5
mt5.initialize()

# Define the symbol and lot size
symbol = "EURUSD.a"
lot = 0.01

# Get the ask price for the symbol
ask = mt5.symbol_info(symbol).ask

# Choose the deviation (slippage) in points
deviation = 100

# Find the filling mode for the symbol
def find_filling_mode(symbol):
    
        for i in range(2):
            request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}

            result = mt5.order_check(request)
            
            if result.comment == "Done":
                break

        return i

# Create the trade request dictionary
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": ask,
    "deviation": deviation,
    "type_filling": 1,
    "type_time": mt5.ORDER_TIME_GTC,
}

# Send the trade request and check the result
result = mt5.order_send(request)
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("Order failed: ", result.comment)
else:
    print("Order placed: ", result.order)

Order placed:  71001413


In [10]:
mt5.last_error()

(1, 'Success')

In [19]:
mt5.positions_get()

(TradePosition(ticket=71001705, time=1677588241, time_msc=1677588241051, time_update=1677588241, time_update_msc=1677588241051, type=0, magic=0, identifier=71001705, reason=3, volume=0.01, price_open=1.06099, sl=0.0, tp=0.0, price_current=1.06071, swap=0.0, profit=-0.42, symbol='EURUSD.a', comment='', external_id=''),)

In [20]:
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "position": 71001705,
    "type": mt5.ORDER_TYPE_SELL,
    "price": mt5.symbol_info(symbol).bid,
    "deviation": deviation,
    "type_filling": find_filling_mode(symbol),
    "type_time": mt5.ORDER_TIME_GTC,
}
mt5.order_send(request)

OrderSendResult(retcode=10009, deal=55607386, order=71001724, volume=0.01, price=1.06081, bid=0.0, ask=0.0, comment='Request executed', request_id=10, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD.a', volume=0.01, price=1.06081, stoplimit=0.0, sl=0.0, tp=0.0, deviation=100, type=1, type_filling=1, type_time=0, expiration=0, comment='', position=71001705, position_by=0))

Automate Buy and Sell orders

In [21]:
def send_order(symbol, lot, buy, sell, id_position=None, comment=" No specific comment", magic=0):
    mt5.initialize()

    def find_filling_mode(symbol):
        for i in range(2):
            request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}
            result = mt5.order_check(request)
            if result.comment == "Done":
                break

        return i
    
    if buy and id_position==None:
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info(symbol).ask,
            "magic": magic,
            "comment": comment,
            "type_filling": find_filling_mode(symbol),
            "type_time": mt5.ORDER_TIME_GTC,
        }
        result = mt5.order_send(request)
        return result
    
    if sell and id_position==None:
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info(symbol).bid,
            "deviation": 10,
            "magic": magic,
            "comment": comment,
            "type_filling": find_filling_mode(symbol),
            "type_time": mt5.ORDER_TIME_GTC,
        }
        result = mt5.order_send(request)
        return result
    
    # CLOSE A TRADE

    if buy and id_position!=None:
        request = {
            "position": id_position,
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info(symbol).bid,
            "deviation": 10,
            "magic": magic,
            "comment": comment,
            "type_filling": find_filling_mode(symbol),
            "type_time": mt5.ORDER_TIME_GTC,
        }
        result = mt5.order_send(request)
        return result
    
    if sell and id_position!=None:
        request = {
            "position": id_position,
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info(symbol).ask,
            "deviation": 10,
            "magic": magic,
            "comment": comment,
            "type_filling": find_filling_mode(symbol),
            "type_time": mt5.ORDER_TIME_GTC,
        }
        result = mt5.order_send(request)
        return result


In [22]:
information = send_order("EURUSD.a", 0.01, True, False)
information

OrderSendResult(retcode=10009, deal=55607775, order=71002132, volume=0.01, price=1.0615, bid=0.0, ask=0.0, comment='Request executed', request_id=11, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD.a', volume=0.01, price=1.0615, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=1, type_time=0, expiration=0, comment=' No specific comment', position=0, position_by=0))

In [23]:
send_order("EURUSD.a", 0.01, True, False, id_position=information.order)

OrderSendResult(retcode=10009, deal=55607930, order=71002286, volume=0.01, price=1.06154, bid=0.0, ask=0.0, comment='Request executed', request_id=12, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD.a', volume=0.01, price=1.06153, stoplimit=0.0, sl=0.0, tp=0.0, deviation=10, type=1, type_filling=1, type_time=0, expiration=0, comment=' No specific comment', position=71002132, position_by=0))